In [1]:
import sys 
sys.path.append("../")
import pandas as pd 
import numpy as np 
import os 
import misc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

## Load dataset

In [5]:

frame = pd.read_csv("D:\\LungCancer\\src\\dataset\\SUPPORT\\support2.csv")
frame.head(2)

,sno,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
0,1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,...,1.199951,141.0,7.459961,NaN,NaN,NaN,7.0,7.0,NaN,7.0
1,2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,...,5.500000,132.0,7.250000,NaN,NaN,NaN,NaN,1.0,<2 mo. follow-up,1.0


## Convert text data to categorical value

In [6]:
x1 = frame[['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp', 'temp',
         'bili', 'crea', 'sod', 'charges', 'sps', 'aps', 'surv2m', 'surv6m', 'adlsc']]

catfeats = ['sex', 'dzgroup', 'dzclass', 'race', 'ca']
x2 = pd.get_dummies(frame[catfeats])

t = frame['d.time']
e = frame['death']

x = pd.concat([x1, x2], axis=1)
x

,age,num.co,meanbp,wblc,hrt,resp,temp,bili,crea,sod,...,dzclass_Cancer,dzclass_Coma,race_asian,race_black,race_hispanic,race_other,race_white,ca_metastatic,ca_no,ca_yes
0,62.84998,0,97.0,6.000000,69.0,22.0,36.00000,0.199982,1.199951,141.0,...,1,0,0,0,0,1,0,1,0,0
1,60.33899,2,43.0,17.097656,112.0,34.0,34.59375,NaN,5.500000,132.0,...,0,0,0,0,0,0,1,0,1,0
2,52.74698,2,70.0,8.500000,88.0,28.0,37.39844,2.199707,2.000000,134.0,...,0,0,0,0,0,0,1,0,1,0
3,42.38498,2,75.0,9.099609,88.0,32.0,35.00000,NaN,0.799927,139.0,...,1,0,0,0,0,0,1,1,0,0
4,79.88495,1,59.0,13.500000,112.0,20.0,37.89844,NaN,0.799927,143.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100,66.07300,1,109.0,7.399414,104.0,22.0,35.69531,0.399963,1.099854,131.0,...,0,0,0,0,0,0,1,0,1,0
9101,55.15399,1,43.0,NaN,0.0,8.0,38.59375,NaN,5.899414,135.0,...,0,1,0,0,0,0,1,0,1,0
9102,70.38196,1,111.0,8.398438,83.0,24.0,36.69531,0.399963,2.699707,139.0,...,0,0,0,0,0,0,1,0,1,0
9103,47.01999,1,99.0,7.599609,110.0,24.0,36.39844,0.399963,3.500000,135.0,...,0,0,0,0,0,0,1,0,0,1


In [4]:
remove = ~np.any(pd.isnull(x), axis=1)
print(len(t))
x = x[remove]
t = t[remove]
e = e[remove]

x = pd.DataFrame(np.array(x))
x

9105


,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,62.84998,0.0,97.0,6.000000,69.0,22.0,36.00000,0.199982,1.199951,141.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,52.74698,2.0,70.0,8.500000,88.0,28.0,37.39844,2.199707,2.000000,134.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,62.37097,1.0,78.0,11.699219,120.0,28.0,37.39844,0.399963,1.599854,132.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,42.25897,0.0,84.0,11.298828,94.0,20.0,38.19531,0.199982,0.799927,139.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,47.94498,1.0,70.0,10.898438,105.0,24.0,36.50000,0.500000,1.000000,134.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,68.61597,2.0,71.0,12.599609,110.0,10.0,36.19531,0.599976,0.799927,128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6307,66.07300,1.0,109.0,7.399414,104.0,22.0,35.69531,0.399963,1.099854,131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6308,70.38196,1.0,111.0,8.398438,83.0,24.0,36.69531,0.399963,2.699707,139.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6309,47.01999,1.0,99.0,7.599609,110.0,24.0,36.39844,0.399963,3.500000,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Remove patients with missing data

In [5]:
# x = pd.DataFrame(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x))
# x = StandardScaler().fit_transform(x)

remove = ~np.isnan(t)

x = x[remove]
t = t[remove]
e = e[remove]

c:\users\duyphuong\anaconda3\envs\duyphuong\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [8]:
frame_new = pd.concat([x, t, e], axis=1)
frame_new.to_csv("../dataset/SUPPORT/support_cleaned.csv", index=False)